Let's build the simplest models always predicting a cosnstant for `updrs_1, updrs_2, updrs_3, updrs_4`.

Sometimes constants are not bad baselines :)

In [1]:
import pandas as pd
import numpy as np

import amp_pd_peptide

from scipy.optimize import minimize

In [2]:
def smape_plus_1(y_true, y_pred):
    y_true_plus_1 = y_true + 1
    y_pred_plus_1 = y_pred + 1
    metric = np.zeros(len(y_true_plus_1))
    
    numerator = np.abs(y_true_plus_1 - y_pred_plus_1)
    denominator = ((np.abs(y_true_plus_1) + np.abs(y_pred_plus_1)) / 2)
    
    mask_not_zeros = (y_true_plus_1 != 0) | (y_pred_plus_1 != 0)
    metric[mask_not_zeros] = numerator[mask_not_zeros] / denominator[mask_not_zeros]
    
    return 100 * np.nanmean(metric)

In [3]:
train_clinical_data = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv')
supplemental_clinical_data = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')
train_clinical_all = pd.concat([train_clinical_data, supplemental_clinical_data])

More detailed information why we use only plus 6, 12, 24 months of **y_true** and other mechanics of this competition could be found in the notebook [Explain Dataset, Test API, Cross-Validation Tips](https://www.kaggle.com/code/vitalykudelya/explain-dataset-test-api-cross-validation-tips).

In [4]:
for plus_month in [6, 12, 24]:
    train_shift = train_clinical_all[['patient_id', 'visit_month', 'updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].copy()
    train_shift['visit_month'] -= plus_month
    train_shift.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_{plus_month}' for i in range(1, 5)}, inplace=True)
    train_clinical_all = train_clinical_all.merge(train_shift, how='left', on=['patient_id', 'visit_month'])

train_clinical_all.rename(columns={f'updrs_{i}': f'updrs_{i}_plus_0' for i in range(1, 5)}, inplace=True)
train_clinical_all

,visit_id,patient_id,visit_month,updrs_1_plus_0,updrs_2_plus_0,updrs_3_plus_0,updrs_4_plus_0,upd23b_clinical_state_on_medication,updrs_1_plus_6,updrs_2_plus_6,updrs_3_plus_6,updrs_4_plus_6,updrs_1_plus_12,updrs_2_plus_12,updrs_3_plus_12,updrs_4_plus_12,updrs_1_plus_24,updrs_2_plus_24,updrs_3_plus_24,updrs_4_plus_24
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN,8.0,10.0,34.0,NaN,10.0,10.0,41.0,0.0,16.0,9.0,49.0,0.0
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN,8.0,9.0,30.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN,10.0,10.0,41.0,0.0,7.0,13.0,38.0,0.0,14.0,13.0,49.0,0.0
3,55_9,55,9,8.0,9.0,30.0,0.0,On,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,55_12,55,12,10.0,10.0,41.0,0.0,On,7.0,13.0,38.0,0.0,16.0,9.0,49.0,0.0,17.0,18.0,51.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,65382_0,65382,0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4834,65405_0,65405,0,5.0,16.0,31.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4835,65405_5,65405,5,NaN,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4836,65530_0,65530,0,10.0,6.0,24.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def function_to_minimize(x):
    metric = smape_plus_1(y_true=array, y_pred=np.array([x[0]] * len(array)))
    return metric

target_to_prediciton = {}
for i in range(1, 5):
    target = f'updrs_{i}'
    columns_with_target = [column for column in train_clinical_all.columns if target in column]
    array = train_clinical_all[columns_with_target].values.ravel()
    the_best_constant = minimize(
        fun=function_to_minimize,
        x0=[0]
    ).x[0]
    median = np.nanmedian(array)
    
    print(target)
    print({'the_best_constant': the_best_constant, 'median': median, 'diff': the_best_constant - median})
    print()
    
    target_to_prediciton[target] = np.round(the_best_constant)
    
target_to_prediciton

updrs_1
{'the_best_constant': 5.999999994158496, 'median': 6.0, 'diff': -5.841504169268319e-09}

updrs_2
{'the_best_constant': 5.999999994880736, 'median': 6.0, 'diff': -5.119264123720768e-09}

updrs_3
{'the_best_constant': 22.999999996944712, 'median': 21.0, 'diff': 1.9999999969447124}

updrs_4
{'the_best_constant': 0.0, 'median': 0.0, 'diff': 0.0}



{'updrs_1': 6.0, 'updrs_2': 6.0, 'updrs_3': 23.0, 'updrs_4': 0.0}

In [6]:
# It's possible to find the minimum (not guaranteed) of any other function with this approach
# (replace the function_to_minimize and x can be an array)

`target_to_prediciton` is our "model" which basically consists of the best consant for our train data for each **y_true** (updrs_1, updrs_2, updrs_3, updrs_4).

In [7]:
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files

# The API will deliver four dataframes in this specific order:
for test_clinical_data, test_peptides, test_proteins, sample_submission in iter_test:
    sample_submission['patient_id'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[0]))
    sample_submission['visit_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[1]))
    sample_submission['target_name'] = sample_submission['prediction_id'].map(lambda x: 'updrs_' + x.split('_')[3])
    sample_submission['plus_month'] = sample_submission['prediction_id'].map(lambda x: int(x.split('_')[5]))
    sample_submission['pred_month'] = sample_submission['visit_month'] + sample_submission['plus_month']
    
    for i in range(1, 5):
        target = f'updrs_{i}'
        mask_target = sample_submission['target_name'] == target
        sample_submission.loc[mask_target, 'rating'] = target_to_prediciton[target]
        
    # call the env.predict for every iteration
    env.predict(sample_submission[['prediction_id', 'rating']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


# Why medians are good constants for SMAPE+1?

Metric in this competitions is **SMAPE+1**. Read the update [Evaluation updates scheduled for March 14th [Updates are now live]](https://www.kaggle.com/competitions/amp-parkinsons-disease-progression-prediction/discussion/394534).

$${\rm SMAPE+1} = \frac{100}{n} \sum_{i=1}^n \frac{|(F_i+1) - (A_i+1)|}{(|F_i+1| + |A_i+1|) / 2}, $$
where $A_i$ is the actual value and $F_i$ is the forecast value. It's just **SMAPE** metric with +1 to y_pred and y_true.

We can see that **SMAPE+1** metric is **[MAE](https://en.wikipedia.org/wiki/Mean_absolute_error)** metric in the numerator. 
For the **MAE** metric if we have an array of numbers and want to find a constant with the minimum loss, then we need to choose the **median** of the array (for example for **RMSE** it would be the **mean**). **SMAPE+1** inherits some properties from **MAE** and usually **median** is a good baseline.
